<a href="https://colab.research.google.com/github/jackl14/Carbon-Footprint-Huella-de-Carbono/blob/main/CaluladorDeCO2Espa%C3%B1ol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title
!pip install deep-translator

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.4 MB/s eta 0:00:00


In [ ]:
# @title
#Cargar una biblioteca necesaria para el analisis de datos
import pandas as pd
from deep_translator import GoogleTranslator
import warnings

plastic_data = {
    'General': 3.31,
    'ABS': 3.76,
    'General Polyethylene': 2.54,
    'High Density Polyethylene (HDPE) Resin': 1.93,
    'HDPE Pipe': 2.52,
    'Low Density Polyethylene (LDPE) Resin': 2.08,
    'LDPE Film': 2.60,
    'Nylon (Polyamide) 6 Polymer': 9.14,
    'Nylon (polyamide) 6,6 Polymer': 7.92,
    'Polycarbonate': 7.62,
    'Polypropylene, Orientated Film': 3.43,
    'Polypropylene, Injection Moulding': 4.49,
    'Expanded Polystyrene': 3.29,
    'General Purpose Polystyrene': 3.43,
    'High Impact Polystyrene': 3.42,
    'Thermoformed Expanded Polystyrene': 4.39,
    'Polyurethane Flexible Foam': 4.84,
    'Polyurethane Rigid Foam': 4.26,
    'PVC General': 3.10,
    'PVC Pipe': 3.23,
    'Calendered Sheet PVC': 3.19,
    'PVC Injection Moulding': 3.30,
    'UPVC Film': 3.16
}

insulation_and_other = {
    'General Insulation': 1.86,
    'Cork': 0.19,
    'Fibreglass (Glasswool)': 1.35,
    'Flax (Insulation)': 1.7,
    'Mineral wool': 1.28,
    'Paper wool': 0.63,
    'Rockwool': 1.12,
    'Woodwool (Board)': 0.98,
    'Calcium Silicate Sheet': 0.13,
    'Cotton, Padding': 1.28,
    'Cotton, Fabric': 6.78,
    'Flax': 1.7,
    'Fly Ash': 0.008,
    'Grit': 0.007,
    'Ground Limestone': 0.032,
    'Glass Reinforced Plastic - GRP - Fibreglass': 8.1,
    'Mandolite': 1.4,
    'Mineral Fibre Tile (Roofing)': 2.7,
    'Perlite - Expanded': 0.52,
    'Perlite - Natural': 0.03,
    'Quartz powder': 0.023,
    'Shingle': 0.3,
    'Slag (GGBS)': 0.083,
    'Straw': 0.01,
    'Terrazzo Tiles': 0.118,
    'Vermiculite - Expanded': 0.52,
    'Vermiculite - Natural': 0.032,
    'Water': 0.0008,
    'General (Gypsum)': 0.13,
    'Plasterboard': 0.39
}

#Ignorar los avisos de excel
warnings.simplefilter("ignore", category=UserWarning)

#Añadir los datos de Circular Ecology
xls = pd.ExcelFile('ICE DB V3.0 - 10 Nov 2019.xlsx')

#Traducir
eng_esp_dict = {"Plastic": "Plástico",
                "Insulation and Other": "Aislamiento y Otro",
                "Concrete": "Hormigón",
                "Aggregates_Sand": "Agregados y arena",
                "Bitumen": "Bitumen",
                "Cement_and_Mortar": "Cemento y argamasa",
                "Clay_Bricks": "Ladrillo"
                #Acá puedes añadir otras frases

                }


def translate_to_spanish(text):
  try:
    if text in eng_esp_dict:

      return eng_esp_dict[text]

    else:
      translated = GoogleTranslator(source='en', target='es').translate(text)  # Use deep-translator
      #print(translated)
      #translation = translated.text
      translation = translated
      if translation == 'Vaso':
        translation_final = 'Vidrio'
      elif 'Timber' in translation:
        translation_final = translation.replace('Timber', 'Madera')
      elif translation == 'Agregados_sand':
        translation_final = 'Agregados y arena'
      elif translation == 'Cemento_and_mortar':
        translation_final = 'Cemento y argamasa'
      elif 'Ash Siliceous' in translation:
        translation_final = translation.replace('Ash Siliceous', 'Ceniza de Sílice')

      else:
        translation_final = translation

      eng_esp_dict[text] = translation_final

      return translation_final

  except:
    return text


  #Función que ayudará con navigar la hoja de calculo
def get_value_to_right(df, column_name, name):
      try:
          # Find the index of the row where 'Name' matches the given name
          index = df.index[df['Unnamed: 5'] == name][0]

          # Get the value in the column to the right
          value_to_right = df.loc[index, column_name]

          assert value_to_right is int or float
          return value_to_right
      except IndexError:
          return None  # Handle case where name is not found in dataframe

#Función para borrar entradas indeseadas
def remove_entries_by_value(value, list1, list2, list3):
    try:
        idx = list1.index(value)  # Find index of 'value' in list1
        del list1[idx]            # Remove entry from list1 at index idx
        del list2[idx]            # Remove entry from list2 at index idx
        del list3[idx]            # Remove entry from list3 at index idx
    except ValueError:
        print(f"'{value}' no se encontró en {list1}.")


def select_material():
  #Elegir Material

  #Generar un menu de opciones posibles de tipos de materiales
  sheet_names = xls.sheet_names
  sheet_list = []
  for entry in sheet_names:
    if entry != 'Concrete':
      sheet_list.append(entry)
  materials = sheet_list[10:19]
  materials.append("Plastic")
  materials.append("Insulation and Other")
  materials_esp = []
  for entry in materials:
    materials_esp.append(translate_to_spanish(entry))

  #Pedir un tipo de material de usuario
  print("Acá hay los tipos de materiales posibles que puedes añadir: ")
  for entry in materials_esp:
    print(entry)
  selection = input("Entra el material que quieres añadir: ")
  if selection not in materials_esp:
    print("No entraste una seleción valida, intenta otra vez por favor.")
    select_material()
  else:
    selection_eng = materials[materials_esp.index(selection)]
    return selection_eng

#select_material()




#Buscar el tipo de material especifico que el usuario quiere
def search_material(selection):

  if selection == "Plastic":
    final_list = plastic_data.keys()
  elif selection == "Insulation and Other":
    final_list = insulation_and_other.keys()

  else:


    #Llenar espacios vacios
    df_raw = pd.read_excel('ICE DB V3.0 - 10 Nov 2019.xlsx', sheet_name=selection)
    df = df_raw.fillna("Empty")

    #Sacar el punto de empieza y de final
    for i in range(len(df)):
      location = df.loc[i,"Unnamed: 5"]
      if location == '<<< Start of Main Embodied Carbon Data >>>':
        start = i
      elif location == '<<< End of Main Embodied Carbon Data >>>':
        endpt = i

    list_of_interest = df['Unnamed: 5'][start:endpt].tolist()

    #Seperar los materiales utiles del otro texto
    clean_list = []
    keep_going = True
    i = 2

    while keep_going:
        point = list_of_interest[i]
        #print(point)
        if point == "Empty" and all(item == "Empty" for item in list_of_interest[i+1:i+4]):
            break  # Break the loop if current and next three entries are all "Empty"
        clean_list.append(point)
        i += 1

    final_list = []


    if selection == 'Aggregates_Sand':
      words = ['Aggregates and sand']
    elif selection == 'Bitumen':
      words = ["Bitumen", "bitumen"]
    elif selection == "Cement_and_Mortar":
      words = ["CEM", "Mortar ", "admixtures"]
    elif selection == 'Clay_Bricks':
      words = ['Clay']
    else:
      words = [selection]

    for word in words:
      for item in clean_list:
        if word in str(item) and 'World Aluminum' not in item:
          final_list.append(item)

  unique_final = []
  [unique_final.append(x) for x in final_list if x not in unique_final]

  unique_final_esp = []

  for i in unique_final:
    esp_i = translate_to_spanish(i)
    unique_final_esp.append(esp_i)
    print(esp_i)

#Selecionar el material y buscar su huella de carbono
  try:
    material_selection = input("Entra el material específico que quieres usar:")
    material_selection_eng = unique_final[unique_final_esp.index(material_selection)]

    if selection != 'Plastic' and selection != "Insulation and Other":
      carbon_footprint_str = get_value_to_right(df, 'Unnamed: 6', material_selection_eng)
      carbon_footprint = float(carbon_footprint_str)

    elif selection == "Plastic":
      carbon_footprint = plastic_data[material_selection_eng]
    else:
      carbon_footprint = insulation_and_other[material_selection_eng]

    return material_selection, carbon_footprint

  except Exception as e:
    print("Pareces haber entrado incorectamente a algo, o tal vez no hay información sobre el material que elegiste.")
#search_material('Cement_and_Mortar')


def add_material():
  print()
  selection = select_material()
  print()
  print("Ahora escoge el tipo específico. Acá hay sus opciones:")
  print()
  try:
    material, footprint = search_material(selection)
  except:
    print("Una huella de carbono para tu elección no se pudo encontrar. Quizás intenta con un material similar.")
    return

  print()
  print("Ahora entra la cantidad del material que quieres usar. Supón que es en kg a menos que se diga algo diferente.")
  quantity_str = input("Cantidad: ")
  try:
    quantity = float(quantity_str)
    print()
    emmisions = footprint*quantity
    print(f'El total de emisiones incorporadas para {quantity} unidades de {material} de {emmisions} kg de CO2.')
    return material, emmisions, quantity
  except:
    print("No entraste un número valido.")

def main():
  print("Bienvenidos a la herramienta de la calculación de la huella de carbono para un proyecto.")
  print()
  materials = []
  emmisions = []
  quantities = []

  while True:
    print()
    print("Para añadir otro material, entra a.")
    print("Para ver un resumen de tu proyecto, entra r.")
    print("Para borrar un material, entra x.")
    print("Para dejar de usar la herramienta, entra d.")
    print()
    choice = input("Tu decisión: ")
    print()

    if choice == 'a':
      try:
        material, emmision, quantity = add_material()
        materials.append(material)
        emmisions.append(emmision)
        quantities.append(quantity)

        print("Añadido con éxito")
      except:
        print("No entraste información valida.")

    elif choice == 'r':

      df = pd.DataFrame({
      'Material': materials,
      'Cantidad (kg a menos que se diga lo contrario)': quantities,
      'Emisiones (kg CO2)': emmisions
        })
      # Set 'Material' as the index
      df.set_index('Material', inplace=True)

      # Calculate totals for integer columns
      totals = {}
      for col in df.columns:
          if pd.api.types.is_numeric_dtype(df[col]):  # Check if column is numeric (integer or float)
              totals[col] = df[col].sum()

      # Convert totals to DataFrame row
      totals_df = pd.DataFrame(totals, index=['Total'])

      # Concatenate totals row to the original DataFrame
      df = pd.concat([df, totals_df])
      print(df)

      save = input("¿Quieres descargar esta hoja en tu computadora? (si/no): ")
      if save == 'si':

        name = input("Entra un nombre para este archivo: ")
        print('Es necesario hacer clic en el botón circular con el retangulo adentro para acarbar con la descarga.')
        # Save DataFrame to CSV in Colab
        csv_file_path = '/content/sample_data/' + name + '.csv'  # Change path as needed
        df.to_csv(csv_file_path, index=True)

        # Download CSV file from Colab
        from google.colab import files
        files.download(csv_file_path)

    elif choice == 'x':
      print("Acá hay los materiales que actualmente tienes")
      for item in materials:
        print(item)
      print()
      removed = input("Entra el nombre del artículo que quieres borrar: ")
      remove_entries_by_value(removed, materials, quantities, emmisions)
      print(f'{removed} se borró con exito.')
      print()

    elif choice == 'd':
      break

main()


Bienvenidos a la herramienta de la calculación de la huella de carbono para un proyecto.


Para añadir otro material, entra a.
Para ver un resumen de tu proyecto, entra r.
Para borrar un material, entra x.
Para dejar de usar la herramienta, entra d.

Tu decisión: a


Acá hay los tipos de materiales posibles que puedes añadir: 
Agregados y arena
Aluminio
Asfalto
Bitumen
Cemento y argamasa
Ladrillo
Vidrio
Acero
Madera
Plástico
Aislamiento y Otro
Entra el material que quieres añadir: Ladrillo

Ahora escoge el tipo específico. Acá hay sus opciones:

Ladrillo de arcilla
Ladrillo de arcilla, un ladrillo, 215 x 65 x 102,5 mm
Muros de ladrillos de arcilla - por m2 de muro incluyendo mortero, sin incluir residuos
Entra el material específico que quieres usar:Ladrillo de arcilla

Ahora entra la cantidad del material que quieres usar. Supón que es en kg a menos que se diga algo diferente.
Cantidad: 300

El total de emisiones incorporadas para 300.0 unidades de Ladrillo de arcilla de 63.9 kg de CO

KeyboardInterrupt: Interrupted by user